In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory (Auditbot_backend) to the system path
sys.path.append(
    os.path.abspath(
        os.path.join(
            os.path.dirname(f"{os.getcwd()}/hyde.ipynb"),
            '..'
        )
    )
)

# import custom modules
from utils.preprocessing import *
from utils.json_parser import *
from utils.content_page_parser import *
from utils.retriever import *
from utils.custom_print import *
from utils.prompt_engineering import *
from utils.db_utils import *
from utils.langsmith_trace import *
from utils.initialisations import *

In [3]:
# RAG input parameters =======================================================

# question = "What are the findings pertaining to grant?"
# query = "extract finding on weakness in access controls from FY2018/19 to FY2020/21 AGO's report."
question = "extract finding on weakness in access controls from FY2018/19 to FY2020/21 AGO's report. Tabulate the output with row heading as Year of Report and details of findings."

# HYPERPARAMETERS ============================================================

# preprocessing --------------------------------------------------------------

# Chunk into sentences ('s') or paragraphs ('p')
chunking='p' 

# Group smaller chunks into a bigger chunk
grouping=1

# control minimum chubk size
min_chunk_size=100

# vector store ---------------------------------------------------------------

# add to data base in batches
batch_size = 1000

# Ranking --------------------------------------------------------------------

# top k matches for ranking. 
# Both sparse and dense search find top_k matches so hybrid search will return 
# at least top_k matches and most 2 * top_k matches
top_k = 30

# weights for each retrieval for reciprocal rank fusion
weights = [0.5, 0.5]

# reciprocal ranking fusion constant
k = 60

# Reranking ------------------------------------------------------------------

# top n matches for reranking
top_n = 20

# Cross encoder model
# claimed to be deprecated because it is bad but seems to still work fine
# model_name = "cross-encoder/stsb-roberta-base"

# best performing on Microsoft tests
model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2"


# IMPROVEMENTS ===============================================================
HyDE = True

comments = "This is using HyDE"
# pack parameters for tracing ================================================
params = (question, chunking, grouping, min_chunk_size, batch_size, top_k, 
          weights, k, top_n, model_name, HyDE, comments)

In [4]:
# RUN ONCE
# generate all required data structures

# generate chunks
generate_chunks(DOCUMENT_DIR,
                chunks_path,
                chunk_pageNum_pairs_path,
                s_p_pairs_path, 
                chunking, 
                grouping, 
                min_chunk_size,
                DOC_IDENTIFIER)


# generate inverted tree
has_content_page = True
generate_inverted_tree(chunk_pageNum_pairs_path, 
                       has_content_page, 
                       save_inverted_tree_path,
                       tree_path)

2.595762014389038 seconds
number of chunks: 5922


In [5]:
# RUN ONCE
# retrieve all required data structures

# load tree
inverted_tree = json_file_to_dict(save_inverted_tree_path)

# load chunks from tree's keys
chunks = list(inverted_tree.keys())
print("Number of unique chunks:", len(chunks))

# prepare metadata for chromadb
pre_metadata = list(inverted_tree.values())
metadata = chroma_preprocess_metadata(pre_metadata)

# load sentence paragraph pairs
if (chunking == 's' or chunking == 'f') and grouping == 1:
    print("s_p_pairs will be filled")
    s_p_pairs = json_file_to_dict(s_p_pairs_path)
else:
    s_p_pairs = {}

Number of unique chunks: 5296


In [6]:
# RUN ONCE
# set up vector database for dense embedding search

# chromadb supported model
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )

# create db
client_dense = chromadb.PersistentClient(path="../data/db", 
                                   settings = Settings(allow_reset=True))

# chromadb's embedding function needs streaming
collection = chroma_get_or_create_collection(client_dense, 
                                             name = "audit", 
                                             embedding_function = openai_ef, 
                                             reset = True)

# fill db
chroma_fill_db(collection, chunks, metadata, batch_size)
print("number of embeddings in database:",collection.count())

number of embeddings in database: 5296


In [7]:
# RUN ONCE
# set up elasticseach for sparse embedding search

# create docker container for elasticsearch on terminal/shell
'''
docker run -p 127.0.0.1:9200:9200 -d --name elasticsearch --network elastic-net \
  -e ELASTIC_PASSWORD=$ELASTIC_PASSWORD \
  -e "discovery.type=single-node" \
  -e "xpack.security.http.ssl.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.14.3
'''

# connect to the Elasticsearch cluster from python elasticsearch client
client_sparce = Elasticsearch(
    LOCAL_HOST_URL,
    basic_auth=HTTP_AUTH
)
# checks if client is connected to docker container
print(client_sparce.info(http_auth=HTTP_AUTH))

# index chunks using elasticsearch (saved in docker)
index_elastic_db(client_sparce, index_name, HTTP_AUTH, chunks, reset = True)

{'name': '50cd72118574', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'l-puMSQDTvqL7nQDfYreOg', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
reset index: chromadb_documents


In [8]:
if HyDE:
    query = gpt4o(hyde(question), llm)

else:
    query = question

During the fiscal years FY2018/19 to FY2020/21, the Auditor General's Office (AGO) identified several weaknesses in access controls across various organizations. Below is a tabulated summary of the findings:

| Year of Report | Details of Findings                                                                                                                                                        |
|----------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| FY2018/19      | The AGO reported inadequate user access reviews. Several systems had users with excessive privileges, increasing the risk of unauthorized access to sensitive information.  |
| FY2019/20      | The AGO found poor password management practices, including the lack of enforcement of strong password policies and frequent password changes, compromising system security.  |
| FY2020/21      | The AGO identified insufficient segregation of duties, where users had conflicting responsibilities that could lead to potential abuse or fraud, and weak monitoring of access. |

These consistent findings over the three fiscal years highlight the ongoing challenges faced by organizations in implementing effective access control measures.

In [9]:
# Can be run multiple times when change in query
# 1st ranking

# dense embeding search 
embedding_results = chromadb_embedding_search(collection, query, top_k)

# perform bm_25 using elasticsearch
bm25_results = bm25_elasticsearch(client_sparce, index_name, HTTP_AUTH, query, top_k)


# RRF
good_chunks  = reciprocal_rank_fusion(bm25_results, 
                                 embedding_results, 
                                 weights, 
                                 k)

# ----------------------------------------------------------------------------
# Reranking
best_chunks, scores = reranking(model_name, good_chunks, query, top_n)

# ----------------------------------------------------------------------------
# prompt engineering
prompt = generate_prompt(question, 
                         inverted_tree, 
                         best_chunks, 
                         chunking, 
                         s_p_pairs)

# with hyde
os.environ["TOKENIZERS_PARALLELISM"] = "false"
@traceable
def rag_pipeline(params, prompt, llm):
    traced_params = trace_params(params) 
    response = gpt4o(prompt, llm)
    return response

response = rag_pipeline(params, prompt, llm)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Below is the tabulated output with findings on weaknesses in access controls from the AGO reports for FY2018/19 to FY2020/21:

| Year       | Details of Findings                                                                                                                                                                                                                                         |
|------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 2018/19    | AGO found weaknesses in IT controls in audits of several public sector entities such as the Ministry of Manpower (MOM), Singapore Customs (Customs), and the Ministry of Defence (MINDEF). These included inadequate logging and review of activities of privileged users, such as operating system (OS) and database (DB) administrators, and lapses in the management of user access rights. AGO noted that some of these privileged users were staff of IT vendors (AGO 2018_19, p. 5). Additionally, the weaknesses included inadequate monitoring and review of privileged users’ activities in IT systems, and lapses in the management of user access rights such as not granting access on a needs-only basis (AGO 2018_19, p. 4). Another instance included a check of controls over user accounts and access rights for two of MUIS’ IT systems, revealing weaknesses in management of access rights and conflicting access rights assigned to certain users. MUIS did not conduct annual reviews of access rights for certain IT systems (AGO 2018_19, p. 41). |
| 2019/20    | AGO’s audit of the Enterprise Singapore Board found weaknesses in controls which increased the risk of unauthorized activities and changes not being detected. These weaknesses could compromise the integrity of the ESGP and the grant data within (AGO 2019_20, p. 80).                                                                                                                                                                                                                                                                                                  |
| 2020/21    | Weaknesses in IT controls were noted at the Accountant-General’s Department (AGD) and the Accounting and Corporate Regulatory Authority (ACRA) over the management of the most privileged operating system (OS) user accounts. These accounts have full access privileges, including the ability to make changes to audit logs and security settings. Unauthorized activity could compromise the servers and affect the processing and recording of financial transactions (AGO 2020_21, p. 6). AGO’s audit of the IT general controls relating to NFS@Gov found weak controls over the most privileged OS user accounts and weaknesses in the logging and review of privileged user activities, thus potentially compromising controls over financial transactions (AGO 2020_21, p. 17).                                                                                                            |

Sources:
- AGO 2018_19, OVERVIEW, SUMMARY, Pages: 4-5, 41
- AGO 2019_20, PART III: THEMATIC AUDIT, MINISTRY OF TRADE AND INDUSTRY, Page: 80
- AGO 2020_21, CONTENTS, OVERVIEW, SUMMARY, Pages: 6, 17

In [23]:
# just need to run this again to test
# ----------------------------------------------------------------------------
# prompt engineering
prompt = generate_prompt(question, 
                         inverted_tree, 
                         best_chunks, 
                         chunking, 
                         s_p_pairs)

# with hyde
os.environ["TOKENIZERS_PARALLELISM"] = "false"
@traceable
def rag_pipeline(params, prompt, llm):
    traced_params = trace_params(params) 
    response = gpt4o(prompt, llm)
    return response

response = rag_pipeline(params, prompt, llm)

Below are the findings on weaknesses in access controls from AGO's report for the fiscal years (FY) 2018/19 to 2020/21, presented in a tabulated format:

| Year of Report | Details of Findings |
|----------------|---------------------|
| 2018/19 | **Ministry of Manpower (MOM), Singapore Customs (Customs), Ministry of Defence (MINDEF)** - Weaknesses in IT controls including inadequate logging and review of activities of privileged users, and lapses in the management of user access rights. Privileged users involved were often staff of IT vendors (AGO, 2018/19, p. 5). |
| 2018/19 | **Ministry of Culture, Community and Youth** - Weaknesses in management of access controls for the Electronic Halal (eHalal) system (AGO, 2018/19, p. 42). |
| 2018/19 | **Ministry of Defence (MINDEF)** - IT vendors granted unrestricted read-access to personnel and payroll information in the Enterprise Human Resource (E-HR) system, contrary to the required controlled access (AGO, 2018/19, p. 21). |
| 2018/19 | **Singapore Customs** - Weaknesses in controls over privileged user accounts and monitoring of privileged users’ activities in the TradeNet and eCustoms systems (AGO, 2018/19, p. 27). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weak controls over the most privileged operating system (OS) user account and monitoring of privileged user activities in NFS@Gov. This affected controls over processing and recording of financial transactions (AGO, 2020/21, p. 17). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weaknesses in logging and review of privileged user activities in NFS@Gov. Insufficient oversight increased the risk of undetected unauthorized modifications to configuration settings, compromising control over financial transactions (AGO, 2020/21, p. 25). |
| 2020/21 | **Prime Minister’s Office, Public Service Division** - Excessive access rights granted to encryption/decryption keys and data files were identified, which could impact the encryption/decryption process (AGO, 2020/21, p. 26). |

### Citations:
- AGO Annual Report FY 2018/19, p. 5, 21, 27, 42.
- AGO Annual Report FY 2020/21, p. 17, 25, 26.

In [30]:
# sentence chunking, question on grants
display_markdown('''
The findings from various audit reports by the Auditor-General’s Office (AGO) of Singapore highlight numerous issues and lapses in the management of grants across different statutory boards and ministries. Below are the key findings from selected reports:

### Ministry of Education
**SkillsFuture Singapore Agency:**
- **Inadequate Monitoring and Checks:** Grants disbursed with various lapses, including inadequate monitoring and eligibility checks, inappropriate claims by training providers, and disallowed funding. The total estimated overpayment amounted to $4,217,500 out of $7,716,400 disbursed (AGO, 2021_22, p. 39).

### Ministry of Finance
**Inland Revenue Authority of Singapore:**
- **Evaluation of Grant Applications:** Need for improvement in evaluating grant applications to ensure proper assessment of eligibility and computation of grant quantum (AGO, 2022_23, p. 63).

### Workforce Singapore Agency
- **Oversight of Programme Partners (PPs):** The audit suggested strengthening oversight of PPs administering grants. Although flexibility in applying grant guidelines is needed, it is crucial to meet baseline requirements and ensure consistency in grant administration (AGO, 2019_20, p. 56, 61).
- **Possible Circumvention of Eligibility Criteria:** Instances were found where grants were awarded by not aggregating similar applications, possibly bypassing more stringent controls required for higher value grants (AGO, 2019_20, p. 65).

### Enterprise Singapore (ESG)
- **Customer-Centric Approach & Inconsistency:** ESG adopted a customer-centric approach to reduce administrative burdens but lacked adequate measures to ensure fair and equitable grant administration, leading to inconsistencies (AGO, 2019_20, p. 58).

### National Research Fund (NRF)
- **Lapses in Grant Administration:** Issues included lack of timely reviews for accountability, inadequacies in verification of fund requests, and failure to recover unutilized grants from recipients (AGO, 2014_15, p. 17; 2017_18, Before first page, pp. 49-56).

### Thematic and Summary Audits
- **General Laxity in Administration:** Across various audits, common weaknesses observed included the failure to ensure the correct amounts of grants disbursed, lack of adherence to grant conditions, and lapses in evaluation, monitoring, and verification processes (AGO, 2014_15, p. 3; 2017_18, pp. 4, 8; 2019_20, pp. 54, 60).

### Specific Recommendations
- **Strengthening Controls:** Recommendations include improving eligibility assessments, ensuring adequate oversight and verification, timely action on underperforming projects, and establishing comprehensive frameworks for grant management (AGO, 2018_19, p. 4; 2019_20, p. 56; 2017_18, pp. 45, 50).

These findings emphasize the need for stricter controls, better monitoring, and consistent frameworks to ensure that public funds are used effectively and for intended purposes.
'''
    , raw = True)


The findings from various audit reports by the Auditor-General’s Office (AGO) of Singapore highlight numerous issues and lapses in the management of grants across different statutory boards and ministries. Below are the key findings from selected reports:

### Ministry of Education
**SkillsFuture Singapore Agency:**
- **Inadequate Monitoring and Checks:** Grants disbursed with various lapses, including inadequate monitoring and eligibility checks, inappropriate claims by training providers, and disallowed funding. The total estimated overpayment amounted to $4,217,500 out of $7,716,400 disbursed (AGO, 2021_22, p. 39).

### Ministry of Finance
**Inland Revenue Authority of Singapore:**
- **Evaluation of Grant Applications:** Need for improvement in evaluating grant applications to ensure proper assessment of eligibility and computation of grant quantum (AGO, 2022_23, p. 63).

### Workforce Singapore Agency
- **Oversight of Programme Partners (PPs):** The audit suggested strengthening oversight of PPs administering grants. Although flexibility in applying grant guidelines is needed, it is crucial to meet baseline requirements and ensure consistency in grant administration (AGO, 2019_20, p. 56, 61).
- **Possible Circumvention of Eligibility Criteria:** Instances were found where grants were awarded by not aggregating similar applications, possibly bypassing more stringent controls required for higher value grants (AGO, 2019_20, p. 65).

### Enterprise Singapore (ESG)
- **Customer-Centric Approach & Inconsistency:** ESG adopted a customer-centric approach to reduce administrative burdens but lacked adequate measures to ensure fair and equitable grant administration, leading to inconsistencies (AGO, 2019_20, p. 58).

### National Research Fund (NRF)
- **Lapses in Grant Administration:** Issues included lack of timely reviews for accountability, inadequacies in verification of fund requests, and failure to recover unutilized grants from recipients (AGO, 2014_15, p. 17; 2017_18, Before first page, pp. 49-56).

### Thematic and Summary Audits
- **General Laxity in Administration:** Across various audits, common weaknesses observed included the failure to ensure the correct amounts of grants disbursed, lack of adherence to grant conditions, and lapses in evaluation, monitoring, and verification processes (AGO, 2014_15, p. 3; 2017_18, pp. 4, 8; 2019_20, pp. 54, 60).

### Specific Recommendations
- **Strengthening Controls:** Recommendations include improving eligibility assessments, ensuring adequate oversight and verification, timely action on underperforming projects, and establishing comprehensive frameworks for grant management (AGO, 2018_19, p. 4; 2019_20, p. 56; 2017_18, pp. 45, 50).

These findings emphasize the need for stricter controls, better monitoring, and consistent frameworks to ensure that public funds are used effectively and for intended purposes.


In [26]:
# sentence chunking, question on weakness
display_markdown('''
To address your request, the context provided includes audit reports from different fiscal years and indicates weaknesses in the management of grants. Here are the relevant findings from FY2018/19 to FY2020/21 taken from the AGO's reports, including access control weaknesses and other related issues:

| Year of Report | Details of Findings                                                                                  |
|----------------|------------------------------------------------------------------------------------------------------|
| 2018/19        | AGO noted the need to strengthen controls in the management of social grant programs, especially in the timeliness of approvals and verification of disbursements (AGO 2018/19, p.4). |
| 2019/20        | AGO observed systemic weaknesses and inconsistencies in the administration of grants by PPs on behalf of WSG and ESG and recommended improvement in oversight, documentation, and process consistency (AGO 2019/20, pp.56, 58). |
| 2019/20        | AGO highlighted inadequacies in grant management, including the lack of proper aggregation of grant applications leading to potential circumvention of higher-level scrutiny and approval for significant sums (AGO 2019/20, p.65). |
| 2020/21 (2021/22 Report)| AGO identified lapses in the SkillsFuture Singapore Agency’s management of grants including inadequate monitoring of grant eligibility, improper checks by service providers, and disbursement for ineligible individuals and companies (AGO 2021/22, p.39). |

While the provided contexts from FY2018/19 to FY2020/21 do not directly mention weaknesses in access controls, they do highlight various controls deficiencies and systemic weaknesses in the broader context of grant management. For more detailed access control-related findings, further investigation into specific sections or additional documents might be required.             
''', raw = True)
print("----------------------------------------------------------------------")
# 2nd run, sentence chunking, question on weakness
display_markdown('''
Certainly, here is a table summarizing the findings on weaknesses in access controls from the Auditor-General's Office (AGO) of Singapore's reports from FY2018/19 to FY2020/21:

| **Year of Report** | **Details of Findings**                                                                                                                      |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------|
| **2018_19**        | - **Ministry of Defence (MINDEF)**: Weaknesses in the management of access rights to Enterprise Human Resource (E-HR) system. IT vendor staff had unrestricted read-access to personnel and payroll information (p. 21).<br>- **Islamic Religious Council of Singapore (MUIS)**: Weaknesses in management of access rights, including conflicting access rights and lack of annual reviews for four out of five IT systems (p. 41-42).<br>- **General Public Sector Entities**: Inadequate logging and review of privileged user activities and lapses in management of user access rights, including not granting access on a needs-only basis. These issues were prevalent across different entities (p. 4-5).<br>- **Accountant-General’s Department**: Weak controls over the most privileged operating system user account and inadequate logging and review of privileged user activities in NFS@Gov (p. 25). |
| **2019_20**        | - **Workforce Singapore Agency**: Weak IT controls related to grant administration under the Funding Management System (FMS). Vendor staff had excessive privileged access, including six OS administrators and five DB administrators (all vendor staff) (p. 70).<br>- **Enterprise Singapore Board**: Noted weaknesses in IT controls that increase the risk of unauthorized activities and changes not being detected (p. 80).<br>- **Public Utilities Board (PUB)**: Lapses in compliance with IT controls by a partner, including excessive access rights granted to the partner’s vendor, shared administrator accounts, no separation of privileged and non-privileged roles, and no automatic time-outs for user accounts (p. 37). |
| **2020_21**        | - **Ministry of Finance, Accountant-General’s Department**: Weak controls over the most privileged operating system user account and insufficient logging and review of privileged user activities in NFS@Gov (p. 17). |

Please note that the details presented above are derived from the respective reports and represent a summary of the findings related to weaknesses in access controls.
''', raw = True)


To address your request, the context provided includes audit reports from different fiscal years and indicates weaknesses in the management of grants. Here are the relevant findings from FY2018/19 to FY2020/21 taken from the AGO's reports, including access control weaknesses and other related issues:

| Year of Report | Details of Findings                                                                                  |
|----------------|------------------------------------------------------------------------------------------------------|
| 2018/19        | AGO noted the need to strengthen controls in the management of social grant programs, especially in the timeliness of approvals and verification of disbursements (AGO 2018/19, p.4). |
| 2019/20        | AGO observed systemic weaknesses and inconsistencies in the administration of grants by PPs on behalf of WSG and ESG and recommended improvement in oversight, documentation, and process consistency (AGO 2019/20, pp.56, 58). |
| 2019/20        | AGO highlighted inadequacies in grant management, including the lack of proper aggregation of grant applications leading to potential circumvention of higher-level scrutiny and approval for significant sums (AGO 2019/20, p.65). |
| 2020/21 (2021/22 Report)| AGO identified lapses in the SkillsFuture Singapore Agency’s management of grants including inadequate monitoring of grant eligibility, improper checks by service providers, and disbursement for ineligible individuals and companies (AGO 2021/22, p.39). |

While the provided contexts from FY2018/19 to FY2020/21 do not directly mention weaknesses in access controls, they do highlight various controls deficiencies and systemic weaknesses in the broader context of grant management. For more detailed access control-related findings, further investigation into specific sections or additional documents might be required.             



Certainly, here is a table summarizing the findings on weaknesses in access controls from the Auditor-General's Office (AGO) of Singapore's reports from FY2018/19 to FY2020/21:

| **Year of Report** | **Details of Findings**                                                                                                                      |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------|
| **2018_19**        | - **Ministry of Defence (MINDEF)**: Weaknesses in the management of access rights to Enterprise Human Resource (E-HR) system. IT vendor staff had unrestricted read-access to personnel and payroll information (p. 21).<br>- **Islamic Religious Council of Singapore (MUIS)**: Weaknesses in management of access rights, including conflicting access rights and lack of annual reviews for four out of five IT systems (p. 41-42).<br>- **General Public Sector Entities**: Inadequate logging and review of privileged user activities and lapses in management of user access rights, including not granting access on a needs-only basis. These issues were prevalent across different entities (p. 4-5).<br>- **Accountant-General’s Department**: Weak controls over the most privileged operating system user account and inadequate logging and review of privileged user activities in NFS@Gov (p. 25). |
| **2019_20**        | - **Workforce Singapore Agency**: Weak IT controls related to grant administration under the Funding Management System (FMS). Vendor staff had excessive privileged access, including six OS administrators and five DB administrators (all vendor staff) (p. 70).<br>- **Enterprise Singapore Board**: Noted weaknesses in IT controls that increase the risk of unauthorized activities and changes not being detected (p. 80).<br>- **Public Utilities Board (PUB)**: Lapses in compliance with IT controls by a partner, including excessive access rights granted to the partner’s vendor, shared administrator accounts, no separation of privileged and non-privileged roles, and no automatic time-outs for user accounts (p. 37). |
| **2020_21**        | - **Ministry of Finance, Accountant-General’s Department**: Weak controls over the most privileged operating system user account and insufficient logging and review of privileged user activities in NFS@Gov (p. 17). |

Please note that the details presented above are derived from the respective reports and represent a summary of the findings related to weaknesses in access controls.


In [29]:
# paragraph chunking, question on weakness
# 2019 did not even pup up on reranking
display_markdown('''
Here are the extracted findings on weaknesses in access controls from the FY2018/19 to FY2020/21 AGO reports:

| Year of Report | Details of Findings                                                                                         |
|----------------|-------------------------------------------------------------------------------------------------------------|
| 2018_19        | **Ministry of Defence (MINDEF)**: Weaknesses in management of access rights to the E-HR system, granting IT vendor staff access to personnel and payroll information, including 73 types of data requiring controlled access (p. 21). |
| 2018_19        | **Ministry of Manpower (MOM), Ministry of Defence (MINDEF), Singapore Customs**: Weaknesses in IT controls included inadequate logging and review of activities of privileged users, and lapses in the management of user access rights, with some privileged users being staff of IT vendors (p. 5). |
| 2018_19        | **Singapore Customs**: Weak controls over privileged user accounts and monitoring of privileged users’ activities in the TradeNet system and eCustoms system (p. 27). |
| 2018_19        | **Ministry of Finance - Accountant-General’s Department (AGD)**: Weaknesses in logging and review of privileged user activities. Privileged accounts had access rights to amend configuration settings, risking undetected unauthorized modifications (p. 25). |
| 2020_21        | **Ministry of Finance - Accountant-General’s Department (AGD)**: Weak IT controls over the most privileged operating system (OS) user account and monitoring of privileged user activities, compromising controls over the processing and recording of financial transactions in NFS@Gov (p. 17). |

This table aggregates the specific findings from the AGO's reports, highlighting issues related to access controls over a period of three financial years.
'''
                 , raw = True)

print("----------------------------------------------------------------------")
# 2nd run # paragraph chunking, question on weakness
display_markdown('''
Below are the findings on weaknesses in access controls from AGO's report for the fiscal years (FY) 2018/19 to 2020/21, presented in a tabulated format:

| Year of Report | Details of Findings |
|----------------|---------------------|
| 2018/19 | **Ministry of Manpower (MOM), Singapore Customs (Customs), Ministry of Defence (MINDEF)** - Weaknesses in IT controls including inadequate logging and review of activities of privileged users, and lapses in the management of user access rights. Privileged users involved were often staff of IT vendors (AGO, 2018/19, p. 5). |
| 2018/19 | **Ministry of Culture, Community and Youth** - Weaknesses in management of access controls for the Electronic Halal (eHalal) system (AGO, 2018/19, p. 42). |
| 2018/19 | **Ministry of Defence (MINDEF)** - IT vendors granted unrestricted read-access to personnel and payroll information in the Enterprise Human Resource (E-HR) system, contrary to the required controlled access (AGO, 2018/19, p. 21). |
| 2018/19 | **Singapore Customs** - Weaknesses in controls over privileged user accounts and monitoring of privileged users’ activities in the TradeNet and eCustoms systems (AGO, 2018/19, p. 27). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weak controls over the most privileged operating system (OS) user account and monitoring of privileged user activities in NFS@Gov. This affected controls over processing and recording of financial transactions (AGO, 2020/21, p. 17). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weaknesses in logging and review of privileged user activities in NFS@Gov. Insufficient oversight increased the risk of undetected unauthorized modifications to configuration settings, compromising control over financial transactions (AGO, 2020/21, p. 25). |
| 2020/21 | **Prime Minister’s Office, Public Service Division** - Excessive access rights granted to encryption/decryption keys and data files were identified, which could impact the encryption/decryption process (AGO, 2020/21, p. 26). |

### Citations:
- AGO Annual Report FY 2018/19, p. 5, 21, 27, 42.
- AGO Annual Report FY 2020/21, p. 17, 25, 26.
''', raw = True)




Here are the extracted findings on weaknesses in access controls from the FY2018/19 to FY2020/21 AGO reports:

| Year of Report | Details of Findings                                                                                         |
|----------------|-------------------------------------------------------------------------------------------------------------|
| 2018_19        | **Ministry of Defence (MINDEF)**: Weaknesses in management of access rights to the E-HR system, granting IT vendor staff access to personnel and payroll information, including 73 types of data requiring controlled access (p. 21). |
| 2018_19        | **Ministry of Manpower (MOM), Ministry of Defence (MINDEF), Singapore Customs**: Weaknesses in IT controls included inadequate logging and review of activities of privileged users, and lapses in the management of user access rights, with some privileged users being staff of IT vendors (p. 5). |
| 2018_19        | **Singapore Customs**: Weak controls over privileged user accounts and monitoring of privileged users’ activities in the TradeNet system and eCustoms system (p. 27). |
| 2018_19        | **Ministry of Finance - Accountant-General’s Department (AGD)**: Weaknesses in logging and review of privileged user activities. Privileged accounts had access rights to amend configuration settings, risking undetected unauthorized modifications (p. 25). |
| 2020_21        | **Ministry of Finance - Accountant-General’s Department (AGD)**: Weak IT controls over the most privileged operating system (OS) user account and monitoring of privileged user activities, compromising controls over the processing and recording of financial transactions in NFS@Gov (p. 17). |

This table aggregates the specific findings from the AGO's reports, highlighting issues related to access controls over a period of three financial years.


----------------------------------------------------------------------



Below are the findings on weaknesses in access controls from AGO's report for the fiscal years (FY) 2018/19 to 2020/21, presented in a tabulated format:

| Year of Report | Details of Findings |
|----------------|---------------------|
| 2018/19 | **Ministry of Manpower (MOM), Singapore Customs (Customs), Ministry of Defence (MINDEF)** - Weaknesses in IT controls including inadequate logging and review of activities of privileged users, and lapses in the management of user access rights. Privileged users involved were often staff of IT vendors (AGO, 2018/19, p. 5). |
| 2018/19 | **Ministry of Culture, Community and Youth** - Weaknesses in management of access controls for the Electronic Halal (eHalal) system (AGO, 2018/19, p. 42). |
| 2018/19 | **Ministry of Defence (MINDEF)** - IT vendors granted unrestricted read-access to personnel and payroll information in the Enterprise Human Resource (E-HR) system, contrary to the required controlled access (AGO, 2018/19, p. 21). |
| 2018/19 | **Singapore Customs** - Weaknesses in controls over privileged user accounts and monitoring of privileged users’ activities in the TradeNet and eCustoms systems (AGO, 2018/19, p. 27). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weak controls over the most privileged operating system (OS) user account and monitoring of privileged user activities in NFS@Gov. This affected controls over processing and recording of financial transactions (AGO, 2020/21, p. 17). |
| 2020/21 | **Ministry of Finance, Accountant-General’s Department** - Weaknesses in logging and review of privileged user activities in NFS@Gov. Insufficient oversight increased the risk of undetected unauthorized modifications to configuration settings, compromising control over financial transactions (AGO, 2020/21, p. 25). |
| 2020/21 | **Prime Minister’s Office, Public Service Division** - Excessive access rights granted to encryption/decryption keys and data files were identified, which could impact the encryption/decryption process (AGO, 2020/21, p. 26). |

### Citations:
- AGO Annual Report FY 2018/19, p. 5, 21, 27, 42.
- AGO Annual Report FY 2020/21, p. 17, 25, 26.
